In [7]:
# Load libraries
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import sqlite3
import plotly.express as px
import plotly
import lxml
import numpy as np

In [ ]:
# Process history.sqlite Data

# Import data
con = sqlite3.connect("C:/Data_analysis/timsQC/history.sqlite")
cur = con.cursor()
data = pd.read_sql_query(f"SELECT * FROM InstrumentParameterHistory", con)

data[['Date', 'Time']] = data['DateTime'].str.split('T', expand=True)
# MZ cal
mzCal = data[data['Comment'].str.contains("Calibration of type 'm/z'")]
mzCal[['score']] = mzCal['Comment'].str.extract(fr'{'score'}\s+(.*?)\s+{'and'}')
mzCal['score'] = mzCal['score'].str.strip("'")
mzCal['score'] = mzCal['score'].str.replace("%","",regex=False)
mzCal['score'] = pd.to_numeric(mzCal['score'], errors='coerce')
mzCal[['relative_deviation']] = mzCal['Comment'].str.extract(fr'{'deviation'}\s+(.*?)\s+')
mzCal['relative_deviation'] = mzCal['relative_deviation'].str.strip("'")
mzCal['relative_deviation'] = pd.to_numeric(mzCal['relative_deviation'], errors='coerce')

# Mobility
mobCal = data[data['Comment'].str.contains("Calibration of type 'mobility'")]
mobCal[['score']] = mobCal['Comment'].str.extract(fr'{'score'}\s+(.*?)\s+{'and'}')
mobCal['score'] = mobCal['score'].str.strip("'")
mobCal['score'] = mobCal['score'].str.replace("%","",regex=False)
mobCal['score'] = pd.to_numeric(mobCal['score'], errors='coerce')
mobCal[['relative_deviation']] = mobCal['Comment'].str.extract(fr'{'deviation'}\s+(.*?)\s+')
mobCal['relative_deviation'] = mobCal['relative_deviation'].str.strip("'")
mobCal['relative_deviation'] = pd.to_numeric(mobCal['relative_deviation'], errors='coerce')

# ToF Tuning
tofTuning = data[data['Comment'].str.contains("TofDetectorTuning")]
# Cap cleaning
capCleaning = data[data['Comment'].str.contains("Update capillary cleaning date")]

# Make data wide for heatmaps
mobCal_wide = mobCal.pivot(columns='Date', index='Time', values='score')

In [2]:
# XML importer for ISO-8859-1

import xml.etree.ElementTree as ET

def parse_xml_iso_8859_1(filename):
    """Parses an XML file with ISO-8859-1 encoding."""
    try:
        tree = ET.parse(filename)
        return tree
    except ET.ParseError as e:
        print(f"Error parsing XML: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Example usage:
xml_file = "../old_microTOFQImpacTemAcquisition.method"  # Replace with your file path
tree_element = parse_xml_iso_8859_1(xml_file)
root_element = tree_element.getroot()

In [ ]:
# Imports from .method file

tree=tree_element
root=root_element

CalLists = []
CalVals = []

# transfers XML structure to dataframes for positive calibrations ONLY
for container in root.findall('instrument/qtofimpactemacq/'):
    for element in container.iter('dependent'):
        if element.get('polarity')=='positive':
            # numbers (para_double)
            for item in element.findall('para_double'):
                child_dict = {}
                child_dict['tag'] = item.get('permname')
                child_dict['value'] = item.get('value')
                if item.text:
                    child_dict['text'] = item.text.strip()
                CalVals.append(child_dict)
            # numbers (para_int)
            for item in element.findall('para_int'):
                child_dict = {}
                child_dict['tag'] = item.get('permname')
                child_dict['value'] = item.get('value')
                if item.text:
                    child_dict['text'] = item.text.strip()
                CalVals.append(child_dict)
            # strings (para_string)
            for item in element.findall('para_string'):
                child_dict = {}
                child_dict['tag'] = item.get('permname')
                child_dict['value'] = item.get('value')
                if item.text:
                    child_dict['text'] = item.text.strip()
                CalVals.append(child_dict)
            # lists (para_vec_double)
            for item in element.findall('para_vec_double'):
                for child in item:
                    child_dict = {}
                    child_dict['tag'] = item.get('permname')
                    child_dict['value'] = child.get('value')
                    if child.text:
                        child_dict['text'] = child.text.strip()
                    CalVals.append(child_dict)
df_CalVals=pd.json_normalize(CalVals) # single values

# write raw file 
df_CalVals.to_csv(r'C:/Data_analysis/timsQC/timsCal/old_microTOFQImpacTemAcquisition.txt', header=None, index=None, sep=',', mode='w')


# Internal logic of calibration parameter value selection

Iterate through rows until the 

In [ ]:
#masscal_df=df_CalVals[df_CalVals['tag'].isin(['Calibration_LastCalibrationDate', 'Calibration_Score','Calibration_StdDev','Calibration_StdDevInPPM'])]






In [ ]:
# Params

"LastCalibrationDate" # calibration date (any)
"Calibration_LastCalibrationDate" # mass cal date
"Calibration_Score" # mass cal score
"Calibration_StdDev" # mass cal deviation
"Calibration_StdDevInPPM" # mass cal deviation in PPM

"IMS_Calibration_LastCalibrationDate" # mobility cal date
"IMS_Calibration_Score" # mobility cal score
"IMS_Calibration_StdDev" # mobility cal deviation

"TOF_DetectorTof_LastCalibrationDate" # ToF cal date
"TOF_DeviceReferenceTemp1" # TOF temp 1
"TOF_DeviceReferenceTemp2"  # TOF temp 2

"Quadrupole_Ramping_LastCalibrationDate" # quad ramping cal date

# unassigned
"Calibration_LastCalibrationCurrentMass.entry_double",
"Calibration_LastCalibrationMassError.entry_double",
"Calibration_LastCalibrationMassIntensity.entry_double",

"IMS_Calibration_LastCalibrationReferenceMass.entry_double",
"IMS_Calibration_LastCalibrationMobilityError.entry_double",
"IMS_Calibration_LastCalibrationMassIntensity.entry_double",
"IMS_Calibration_LastCalibrationReferenceMassList",
"IMS_Calibration_LastCalibrationReferenceMobility.entry_double",
"IMS_Calibration_LastCalibrationResultMobility.entry_double",

"Calibration_TOF_CorrectorExtractSetValue",
"Calibration_TOF_CorrectorFillSetValue",
"Calibration_TOF_DeceleratorSetValue",
"Calibration_TOF_DetectorTofSetValue",
"Calibration_TOF_ReflectorSetValue",
"Calibration_TOF_FlightTubeSetValue",
"Calibration_Tof2Score",
"Calibration_Tof2StdDev",
"Calibration_Tof2StdDevInPPM",
"Calibration_Tof2CalC0",
"Calibration_Tof2CalC1",
"Calibration_Tof2CalC2",
"Calibration_TofCalC0",
"Calibration_TofCalC1",
"Calibration_TofCalC2",
"Calibration_Collision_GasSupply_Set",

"Vacuum_LastCapillaryCleaningDate"

In [ ]:
# # TEMP

# tree=tree_element
# root=root_element

# CalLists = []
# CalVals = []

# # transfers XML structure to dataframes
# for container in root.findall('instrument/qtofimpactemacq/'):
#     for element in container.iter('dependent'):
#         # numbers (para_double)
#         for item in element.findall('para_double'):
#             child_dict['tag'] = item.get('permname')
#             child_dict['value'] = item.get('value')
#             #print('attrib')
#             #print(child.attrib)
#             if child.text:
#                 child_dict['text'] = child.text.strip()
#             #val_dict['text'] = child.text.strip()
#             CalVals.append(child_dict)
#         # numbers (para_int)
#         for item in element.findall('para_int'):
#             child_dict = {}
#             child_dict['tag'] = item.get('permname')
#             child_dict['value'] = item.get('value')
#             #print('attrib')
#             #print(child.attrib)
#             if child.text:
#                 child_dict['text'] = child.text.strip()
#             #val_dict['text'] = child.text.strip()
#             CalVals.append(child_dict)
#         # strings (para_string)
#         for item in element.findall('para_string'):
#             child_dict = {}
#             child_dict['tag'] = item.get('permname')
#             child_dict['value'] = item.get('value')
#             #print('attrib')
#             #print(child.attrib)
#             if child.text:
#                 child_dict['text'] = child.text.strip()
#             #val_dict['text'] = child.text.strip()
#             CalVals.append(child_dict)
#         # lists (para_vec_double)
#         for item in element.findall('para_vec_double'):
#             for child in item:
#                 child_dict = {}
#                 child_dict['parameter'] = item.get('permname')
#                 #child_dict['tag'] = child.get('entry_double')
#                 child_dict['value'] = child.get('value')
#                 #child_dict['attributes'] = child.attrib
#                 #child_dict['parent'] = element.attrib
#                 #print('attrib')
#                 #print(child.attrib)
#                 if child.text:
#                     child_dict['text'] = child.text.strip()
#                 CalLists.append(child_dict)
#                 #val_dict = {}
#                 #val_dict['tag'] = item.get('permname')
#                 #val_dict['value'] = item.get('value')
#                 #val_dict['attributes'] = child.attrib
#                 #val_dict['parent'] = element.attrib
#                 #val_dict['text'] = child.text.strip()
#                 #list_of_dicts.append(val_dict)
# df_CalLists=pd.json_normalize(CalLists) # lists from XML
# df_CalVals=pd.json_normalize(CalVals) # single values

In [ ]:
# App Layout

# plotting
app = Dash(__name__)


if __name__ == "__main__":
    app.run(debug=True)
    

In [ ]:
# calplot code temp

scoreplot = calplot(mobCal, x='Date', y="score",  years_title=True, dark_theme=False, showscale = True)
deviationplot = calplot(mobCal, x='Date', y="relative_deviation",  years_title=True, dark_theme=False, showscale = True)
mzscoreplot = calplot(mzCal, x='Date', y="score",  years_title=True, dark_theme=False, showscale = True)
mzdeviationplot = calplot(mzCal, x='Date', y="relative_deviation",  years_title=True, dark_theme=False, showscale = True)

# main
app.layout = html.Div([
    html.H1(children="TIMS ToF Analytics"),
    html.H4('Mobility Scores'),
    dcc.Graph(figure=scoreplot),
    html.H4('Mobility Relative Deviation'),
    dcc.Graph(figure=deviationplot),
    html.H4('MZ Scores'),
    dcc.Graph(figure=mzscoreplot),
    html.H4('MZ Relative Deviation'),
    dcc.Graph(figure=mzdeviationplot)
])

In [ ]:
#previous iteration Code

        # for item in element.findall('para_vec_double'):
        #     for child in item:
        #         child_dict = {}
        #         child_dict['parameter'] = item.get('permname')
        #         #child_dict['tag'] = child.get('entry_double')
        #         child_dict['value'] = child.get('value')
        #         #child_dict['attributes'] = child.attrib
        #         #child_dict['parent'] = element.attrib
        #         #print('attrib')
        #         #print(child.attrib)
        #         if child.text:
        #             child_dict['text'] = child.text.strip()
        #         CalLists.append(child_dict)
        #         #val_dict = {}
        #         #val_dict['tag'] = item.get('permname')
        #         #val_dict['value'] = item.get('value')
        #         #val_dict['attributes'] = child.attrib
        #         #val_dict['parent'] = element.attrib
        #         #val_dict['text'] = child.text.strip()
        #         #list_of_dicts.append(val_dict)